In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!git clone https://github.com/fastai/course-v3
%cd /content/course-v3/nbs/dl2
from exp.nb_08 import *

Cloning into 'course-v3'...
remote: Enumerating objects: 5893, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5893 (delta 0), reused 2 (delta 0), pack-reused 5890
Receiving objects: 100% (5893/5893), 263.10 MiB | 29.57 MiB/s, done.
Resolving deltas: 100% (3251/3251), done.


In [6]:
torch.optim

<module 'torch.optim' from '/usr/local/lib/python3.7/dist-packages/torch/optim/__init__.py'>

## Load dataset and vanila model

In [7]:
path = datasets.untar_data(datasets.URLs.IMAGENETTE_160)

In [8]:
tfms = [make_rgb, ResizeFixed(128), to_byte_tensor, to_float_tensor]
bs = 128
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=2)

In [9]:
nfs = [32, 64, 128, 256]

In [11]:
cbfs = [partial(AvgStatsCallback, accuracy), CudaCallback,
        partial(BatchTransformXCallback, norm_imagenette)]

In [12]:
learn, run= get_learn_run(nfs, data, 0.4, conv_layer, cbs=cbfs)

In [13]:
run.fit(1, learn)

train: [1.794424569648326, tensor(0.3760, device='cuda:0')]
valid: [1.7157151920780256, tensor(0.4456, device='cuda:0')]


---

## Refining the optimizer

In [14]:
import inspect
print(inspect.getsource(compose))

def compose(x, funcs, *args, order_key='_order', **kwargs):
    key = lambda o: getattr(o, order_key, 0)
    for f in sorted(listify(funcs), key=key): x = f(x, **kwargs)
    return x



In [30]:
class Optimizer():
    def __init__(self, params, stepper, **defaults):
        self.param_groups = list(params)
        if not isinstance(self.param_groups[0], list): self.param_groups = [self.param_groups]
        self.hypers = [{**defaults} for p in self.param_groups]
        self.stepper = stepper
    def grad_params(self):
        gps = []
        for pg, hyper in zip(self.param_groups, self.hypers):
            for p in pg:
                if p.grad is not None:
                    gps = gps + [(p, hyper)]
        return gps
    def zero_grad(self):
        for p, hyper in self.grad_params():
            p.grad.detach_()
            p.grad.zero_()
    def step(self):
        for p, hyper in grad_params:
            compose(p, self.stepper, **hyper)

In [31]:
opt = Optimizer(learn.model.parameters(), lambda x : x)

In [32]:
opt.param_groups.__len__()

1